In [1]:
#Copy of comb_sens_gpx.py



#!/usr/bin/env python
# coding: utf-8

# In[1]:

#IMPORTANT: SPECIFY INPUT FILES (LINE 37-39)

#Script takes csv outputs created by "import_data.py" for "Atmotube Pro" data intercepted by "nRF Connect"
#bluetooth API. This data was decoded with "sensor_file_parser.py".
#Then reads gpx data from "TripLogger" mobile app and creates combined pandas dataframe for all sensor and gpx data.

#Long/lat/elevation data is not available for all "time" values, missing data provided using linear 
#interpolation. Location data is only interpolated for up to 5 values between measured points.
#Metereological measurements are taken less frequently by Atmotube Pro - missing data is also interpolated 
#(up to 10 missing values between measurements, linear).

#Final dataframe exported to csv file - one version without ("sensor_gpx_path"), 
#and one with interpolated values ("interp_sensor_gpx_path").



# In[2]:


#imports libraries
import json
import pandas as pd
import gpxpy
import gpxpy.gpx
import numpy as np
from datetime import datetime, timedelta



# In[3]:


#input files - need to be specified before run
sensor_file = '2024-06-09_LL_text-42B0-91CE-4C-0.txt'
gpx_file = '2024-06-09_LL_triplogger-track-20240609T132740+0100.gpx'

#data path specified in import.config.json
with open("import.config.copy.json", "r") as jsonfile:
    import_config = json.load(jsonfile)

env_data_path = import_config["cache_path"] + sensor_file[:-4] + '_env.csv'
meteo_data_path = import_config["cache_path"] + sensor_file[:-4] + '_meteo.csv'
gpx_file_path = import_config["gpx_path"] + gpx_file

#specify output files
sensor_gpx_path = import_config["sens_gpx_path"] + sensor_file[:13] + '_sensor_gpx.csv'
interp_sensor_gpx_path = import_config["sens_gpx_path"] + sensor_file[:13] + '_interp_sensor_gpx.csv'



# In[4]:


#create pandas dataframes from decoded sensor measurements (created by import_data.py)
df_env = pd.read_csv(env_data_path)
df_meteo = pd.read_csv(meteo_data_path)

df_env['time'] = pd.to_datetime(df_env['time'], format='ISO8601') # converts to datetime format to match gpx data
df_meteo['time'] = pd.to_datetime(df_meteo['time'], format='ISO8601')

#add env and meteo data into combined pd dataframe
df_sens_comb = pd.merge_ordered(df_env, df_meteo, fill_method="ffill", left_by='time')



# In[5]:


#read gpx data
with open(gpx_file_path, 'r') as f:
    gpx = gpxpy.parse(f)

# Convert to a dataframe one point at a time.
gpx_points = []
for segment in gpx.tracks[0].segments:
    for p in segment.points:
        gpx_points.append({
            'time': p.time,
            'latitude': p.latitude,
            'longitude': p.longitude,
            'elevation': p.elevation,
        })
df_gpx = pd.DataFrame.from_records(gpx_points)

#Remove +00:00 from the end of each timestamp
gpx_time_str = df_gpx['time']
gpx_time_strip = gpx_time_str.astype(str).str.rstrip('+00:00')
df_gpx['time'] = gpx_time_strip
df_gpx['time'] = pd.to_datetime(df_gpx['time'], format='ISO8601')
df_gpx['time'] = df_gpx['time'] + timedelta(hours=1)



# In[6]:


#add dataframes from sensor and gpx into one file - match the time-steps    
df_sens_gpx = pd.merge_ordered(df_sens_comb, df_gpx, fill_method="ffill", left_by='time')

#df_sens_gpx.to_csv(sensor_gpx_path) #creates csv with only measured points (no interpolation)

#interpolate missing lon/lat/elevation data in gpx files
lat_gpx = df_sens_gpx['latitude']
df_sens_gpx['latitude'] = lat_gpx.interpolate(method='linear', axis=0, limit=5, inplace=False, limit_area='inside')
long_gpx = df_sens_gpx['longitude']
df_sens_gpx['longitude'] = long_gpx.interpolate(method='linear', axis=0, limit=5, inplace=False, limit_area='inside')
ele_gpx = df_sens_gpx['elevation']
df_sens_gpx['elevation'] = ele_gpx.interpolate(method='linear', axis=0, limit=5, inplace=False, limit_area='inside')

#interpolate missing meteo data
#temp_meteo = df_sens_gpx['temperature']
#df_sens_gpx['temperature'] = temp_meteo.interpolate(method='linear', axis=0, limit=10, inplace=False, limit_area='inside')
#hum_meteo = df_sens_gpx['humidity']
#df_sens_gpx['humidity'] = hum_meteo.interpolate(method='linear', axis=0, limit=10, inplace=False, limit_area='inside')
#press_meteo = df_sens_gpx['pressure']
#df_sens_gpx['pressure'] = press_meteo.interpolate(method='linear', axis=0, limit=10, inplace=False, limit_area='inside')
#temp2_meteo = df_sens_gpx['temperature2']
#df_sens_gpx['temperature2'] = temp2_meteo.interpolate(method='linear', axis=0, limit=10, inplace=False, limit_area='inside')


#df_sens_gpx.to_csv(interp_sensor_gpx_path) #creates csv including interpolated values


In [2]:
tree_data_path = '/Users/sebastianherbst/Dissertation/Data processing/Tree data/Borough_tree_list_2021.csv'
tree_data = open(tree_data_path)

df_tree_data = pd.read_csv(tree_data)

/var/folders/gs/8406mnl9255d8h18hpsgdz7w0000gn/T/ipykernel_26150/3823609585.py:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tree_data = pd.read_csv(tree_data)


In [3]:
#Corners of the sample plot = 51.466749,-0.019651; =51.456990,-0.005037

#restrict trees imported to those within a quadrant matching the LL sampling area
df_trees_LL = df_tree_data[(df_tree_data['lat'] >= 51.457000) & (df_tree_data['lat'] <= 51.467000)]
df_trees_LL = df_trees_LL[(df_trees_LL['lon'] >= -0.020000) & (df_trees_LL['lon'] <= -0.005000)]

df_sens_gpx_LL = df_sens_gpx[(df_sens_gpx['latitude'] >= 51.457000) & (df_sens_gpx['latitude'] <= 51.467000)]
df_sens_gpx_LL = df_sens_gpx_LL[(df_sens_gpx_LL['longitude'] >= -0.020000) & (df_sens_gpx_LL['longitude'] <= -0.005000)]
#print(df_trees_LL)

#convert lat long to numpy array
loc_trees_LL_deg = df_trees_LL[['lat','lon']].to_numpy()
loc_air_LL_deg = df_sens_gpx_LL[['latitude','longitude']].to_numpy()

In [4]:
# Approximate radius of earth in km
R = 6378.1

#convert degrees to radians
loc_air_LL = np.radians(loc_air_LL_deg)
loc_trees_LL = np.radians(loc_trees_LL_deg)

#extract lat long from each array
lat_air, lon_air = loc_air_LL[:,0], loc_air_LL[:,1]
lat_trees, lon_trees = loc_trees_LL[:,0], loc_trees_LL[:,1]

#create pairwise differences
dlat = lat_air[:,np.newaxis] - lat_trees
dlon = lon_air[:,np.newaxis] - lon_trees

#apply haversine formula
a = np.sin(dlat / 2)**2 + np.cos(lat_trees) * np.cos(lat_air[:, np.newaxis]) * np.sin(dlon / 2)**2
c = 2 * np.arcsin(np.sqrt(a))

distances = (R * c) *1000 #converts distance from km to m
df_dist = pd.DataFrame(distances)


In [5]:
#get trees within 20m
within_20m = distances <= 20
tree_within_20m = np.where(within_20m)
df_tree_within_20m = pd.DataFrame(tree_within_20m)
df_tree_within_20m = df_tree_within_20m.transpose()

#isolate the numbers (row number in loc_trees_LL_deg of the trees within 20m of air measurements)
lat_deg_trees = df_trees_LL[['lat']].to_numpy()
lon_deg_trees = df_trees_LL[['lon']].to_numpy()
mask = np.zeros(lat_deg_trees.shape, dtype=bool)
index_tree_within_20m = df_tree_within_20m[1].values
mask[index_tree_within_20m] = True

#Extract values <= 20 using the boolean mask
values_within_20 = distances[within_20m]
#combine values and index into pandas dataframe
index_value = list(zip(index_tree_within_20m, values_within_20))
df_index_value = pd.DataFrame(index_value, columns=['Tree number', 'Distance to tree'])
#create unique 'Air-tree ID' which matches each measurement with a tree using unique identifier
df_index_value['Air-tree ID'] = df_tree_within_20m[0].astype(str) + '_' + df_tree_within_20m[1].astype(str)


#add trees within 20m of air pollution measurement (True in mask)
df_loc_trees_LL = pd.DataFrame(loc_trees_LL_deg)
df_loc_trees_LL['Tree within 20m'] = mask

df_loc_trees_LL_index = np.arange(0,len(loc_trees_LL),1)
df_loc_trees_LL['Index (in LL)'] = df_loc_trees_LL_index


In [6]:
#Merge all outputs into single dataframe and export to csv

#merge each air measurement with the specific tree, its lat/long, and distance to air measurement
df_air_tree20 = pd.merge(df_tree_within_20m, df_loc_trees_LL, left_on=1, right_on='Index (in LL)')
df_air_tree20 = df_air_tree20.rename(columns={'0_x': 'Air measurement', '1_x': 'Tree number', '0_y': 'lat', '1_y': 'lon'})
df_air_tree20['Air-tree ID'] = df_air_tree20['Air measurement'].astype(str) + '_' + df_air_tree20['Tree number'].astype(str) #create unique identifier
df_air_tree20 = pd.merge(df_air_tree20, df_index_value, how='left', on='Air-tree ID')

#match the objectid from london tree database with air measurement values within 20m 
df_air_tree20 = pd.merge(df_air_tree20, df_tree_data, how='left', on=['lon', 'lat'])
df_air_tree20 = df_air_tree20.sort_values(by=['Air measurement'])

#organise df
df_air_tree20 = df_air_tree20[[1, 'Air measurement', 'Tree number_x', 'Air-tree ID', 'Distance to tree', 'objectid', 'lat', 'lon', 'Tree within 20m', 'Index (in LL)', 'Tree number_y', 'borough', 'gla_tree_group', 'tree_name', 'taxon_name', 'age', 'age_group', 'spread_m', 'height_m', 'diameter_at_breast_height_cm', 'gdb_geomattr_data', 'load_date', 'updated']]
df_air_tree20 = df_air_tree20.drop([1,'Index (in LL)', 'Tree number_y'], axis=1)

test_airtotree = '/Users/sebastianherbst/Dissertation/Data processing/Test scripts/Test air to tree index.csv'
df_air_tree20.to_csv(test_airtotree)